# Packages 

In [1]:
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import glob
import pathlib
import unicodedata
import tarfile

from pandarallel import pandarallel
pandarallel.initialize()
from unidecode import unidecode

def get_csv_size(csv_name):
    get_size = os.path.getsize(os.getcwd() + '/'+csv_name)
    mb_size = get_size/(1024 * 1024)
    mb_size = round(mb_size,1)
    return mb_size

def get_attribute(filename):
    if filename.endswith('csv'):
        df1 = pd.read_csv(filename,header=[0])
    else:
        df1=pd.read_json(filename,orient="records", lines=True)
    return list(df1.columns) 

def split_large_file(filename, source, output,size=None):
    df1=pd.read_csv(filename,header=[0])
    if size ==None:
        size = get_csv_size(filename)
    num_chunks = size//20
    if num_chunks == 0:
        num_chunks = 1 
    df_all = np.array_split(df1, num_chunks)

    for idx, file in enumerate(df_all):
        file.to_csv('%s/%s_%03d.csv'%(output, source, idx), index=False)   
        
def show_all_files(folder):
    df = pd.DataFrame(glob.glob('%s/*'%folder), columns = ['path'])
    df['root'] = df.path.apply(lambda x: x.split('/')[0])
    df['source'] = df.path.apply(lambda x: x.replace('%s/'%folder,'').split('_')[0])
    df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,'').split('/')[-1])
    df = df.sort_values('source').reset_index(drop=True)
    df['size'] = df['path'].apply(get_csv_size)
    return df

def unicodetoascii(text):
    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")"))
    return TEXT

C:\Users\vikas\AppData\Roaming\Python\Python39\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


# Check csv

In [12]:
pandarallel.initialize(nb_workers=8, progress_bar=True) 
# Place all files to be filtered within the Filter folder, within a subfolder containing the source name
# for example, let there be 3 csv files from El Espectador. 
# The file structure is 
# Filter
# --elespectador
# -- -- elespactador01.csv
# -- -- elespactador02.csv
# -- -- elespactador03.csv


# This creates a dataframe containing a list of all the csv files within the specified folder structure.
df = pd.DataFrame(glob.glob('Filter/*/*.csv'), columns = ['path'])
# This updates the source to be just the first part, the name of the folder.
df['source'] = df.path.apply(lambda x: x.split('/')[2].replace('.csv', ''))
df = df.sort_values('source').reset_index(drop=True)
# Gets the column defenitions of the files and their sizes
df['attributes']= df.path.apply(get_attribute)
df['size']= df.path.apply(get_csv_size)
df

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,attributes,size
0,Filter\nicaragua\nicaragua_articulo66.csv,nicaragua_articulo66,"[Unnamed: 0.1, Unnamed: 0, link, country, news_outlet, title, date, text]",5.4
1,Filter\nicaragua\nicaragua_confidencial.digital.csv,nicaragua_confidencial.digital,"[Unnamed: 0, link, country, news_outlet, title, date, text]",49.1
2,Filter\nicaragua\nicaragua_lajornadanet.csv,nicaragua_lajornadanet,"[Unnamed: 0, link, country, news_outlet, title, date, text]",0.2
3,Filter\nicaragua\nicaragua_laprensani.csv,nicaragua_laprensani,"[Unnamed: 0.1, Unnamed: 0, link, country, news_outlet, title, date, text]",411.7
4,Filter\nicaragua\nicaragua_nicaraguaactual.csv,nicaragua_nicaraguaactual,"[Unnamed: 0, link, country, news_outlet, title, date, text]",21.4
5,Filter\nicaragua\nicaragua_nicaraguainvestiga.csv,nicaragua_nicaraguainvestiga,"[Unnamed: 0, link, country, news_outlet, title, date, text]",21.4


Initialize filter files.

In [43]:
import re


class Trie():
    """Regex::Trie in Python. Creates a Trie out of a list of words. The trie can be exported to a Regex pattern.
    The corresponding Regex should match much faster than a simple Regex union."""

    def __init__(self):
        self.data = {}

    def add(self, word):
        ref = self.data
        for char in word:
            ref[char] = char in ref and ref[char] or {}
            ref = ref[char]
        ref[''] = 1

    def dump(self):
        return self.data

    def quote(self, char):
        return re.escape(char)

    def _pattern(self, pData):
        data = pData
        if "" in data and len(data.keys()) == 1:
            return None

        alt = []
        cc = []
        q = 0
        for char in sorted(data.keys()):
            if isinstance(data[char], dict):
                try:
                    recurse = self._pattern(data[char])
                    alt.append(self.quote(char) + recurse)
                except:
                    cc.append(self.quote(char))
            else:
                q = 1
        cconly = not len(alt) > 0

        if len(cc) > 0:
            if len(cc) == 1:
                alt.append(cc[0])
            else:
                alt.append('[' + ''.join(cc) + ']')

        if len(alt) == 1:
            result = alt[0]
        else:
            result = "(?:" + "|".join(alt) + ")"

        if q:
            if cconly:
                result += "?"
            else:
                result = "(?:%s)?" % result
        return result

    def pattern(self):
        return self._pattern(self.dump())

def trie_regex_from_words(words):
    trie = Trie()
    for word in words:
        trie.add(word)
    return re.compile(r"\b" + trie.pattern() + r"\b", re.IGNORECASE)

In [45]:
# Initialize the relevant keywords and the irrelevant keyword regex's
with open('wiki_relevant_spanish.txt', 'r', encoding='utf8') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = trie_regex_from_words(relevant_rules)

# if relevant_rules.endswith('|\\b'):
#     relevant_rules =relevant_rules[:-3]   

print('relevant_rules:')
print(relevant_rules)

with open('irrelevant_keywords_spanish.txt', 'r', encoding='utf8') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = trie_regex_from_words(irelevant_rules)

# if irelevant_rules.endswith('|\\b'):
#     irelevant_rules =irelevant_rules[:-3]   


print('\nirelevant_rules:')
print(irelevant_rules)

# debugging, no longer relevant
#find_relevant = lambda x: Counter(relevant_rules.findall(str(x).lower()))
#find_exclude = lambda x: Counter(irelevant_rules.findall(str(x).lower()))

relevant_rules:
re.compile('\\b(?:(?:A(?:puñalamiento|suntos\\ (?:exteriores|sociales))|B(?:LOTTER|rexit)|C(?:iberseguridad|ontra(?:\\ el\\ terrorismo|insurgencia))|E(?:mbajada\\ |stados\\ miembros\\ de)|Intergubernamental|P(?:, re.IGNORECASE)

irelevant_rules:
re.compile('\\b(?:(?:A(?:MEX|uto\\.Rac)|Br(?:aves|itish\\.open|oadway)|C(?:BA\\.Playoffs|ar(?:denales|telera)|erveceros|o(?:mpradores|rporat))|D(?:JIA|ow\\.Jones|ólar)|E(?:conomía|xpos)|F(?:I(?:FA\\\\b|NA\\\\b)|, re.IGNORECASE)


In [4]:
# remove all the filtered paths, if they exist.
df['path'] = df['path'].apply(lambda x: x[:-4] + '_filtered.csv')

In [5]:
# Checks for doublely filtered files.
df = df[~df.path.str.contains('_filtered_filtered')]

# Split

In [14]:
# Makes directory for splits. Do not run if a split folder already exists.
cwd = os.getcwd()
os.mkdir(cwd + '/split/')
cwd

'c:\\Users\\vikas\\Documents\\GitHub\\conflibert-spanish\\Filter'

In [15]:
# splits all files above a certain MB limit
pandarallel.initialize(nb_workers=8, progress_bar=True)       
_ = df.apply(lambda x: split_large_file(x['path'],x['source'],'split'), axis=1) 

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [16]:
# Debugging, if there are issues here, let me know.
df = show_all_files('split')
df

,path,root,source,filename,size
0,split\nicaragua_articulo66_000.csv,split\nicaragua_articulo66_000.csv,split\nicaragua,split\nicaragua_articulo66_000.csv,5.4
1,split\nicaragua_laprensani_019.csv,split\nicaragua_laprensani_019.csv,split\nicaragua,split\nicaragua_laprensani_019.csv,20.8
2,split\nicaragua_laprensani_018.csv,split\nicaragua_laprensani_018.csv,split\nicaragua,split\nicaragua_laprensani_018.csv,18.8
3,split\nicaragua_laprensani_017.csv,split\nicaragua_laprensani_017.csv,split\nicaragua,split\nicaragua_laprensani_017.csv,17.4
4,split\nicaragua_laprensani_016.csv,split\nicaragua_laprensani_016.csv,split\nicaragua,split\nicaragua_laprensani_016.csv,17.2
5,split\nicaragua_laprensani_015.csv,split\nicaragua_laprensani_015.csv,split\nicaragua,split\nicaragua_laprensani_015.csv,15.7
6,split\nicaragua_laprensani_014.csv,split\nicaragua_laprensani_014.csv,split\nicaragua,split\nicaragua_laprensani_014.csv,16.1
7,split\nicaragua_laprensani_013.csv,split\nicaragua_laprensani_013.csv,split\nicaragua,split\nicaragua_laprensani_013.csv,16.1
8,split\nicaragua_laprensani_012.csv,split\nicaragua_laprensani_012.csv,split\nicaragua,split\nicaragua_laprensani_012.csv,17.1
9,split\nicaragua_laprensani_011.csv,split\nicaragua_laprensani_011.csv,split\nicaragua,split\nicaragua_laprensani_011.csv,17.1


In [17]:
# Check file sizes.
df['size'].describe()

count    26.000000
mean     19.592308
std       6.952865
min       0.200000
25%      17.100000
50%      18.850000
75%      23.450000
max      36.500000
Name: size, dtype: float64

# step 1  clean

In [18]:
df = show_all_files('split')
# df['attribute'] = df.path.parallel_apply(get_attribute)
df

,path,root,source,filename,size
0,split\nicaragua_articulo66_000.csv,split\nicaragua_articulo66_000.csv,split\nicaragua,split\nicaragua_articulo66_000.csv,5.4
1,split\nicaragua_laprensani_019.csv,split\nicaragua_laprensani_019.csv,split\nicaragua,split\nicaragua_laprensani_019.csv,20.8
2,split\nicaragua_laprensani_018.csv,split\nicaragua_laprensani_018.csv,split\nicaragua,split\nicaragua_laprensani_018.csv,18.8
3,split\nicaragua_laprensani_017.csv,split\nicaragua_laprensani_017.csv,split\nicaragua,split\nicaragua_laprensani_017.csv,17.4
4,split\nicaragua_laprensani_016.csv,split\nicaragua_laprensani_016.csv,split\nicaragua,split\nicaragua_laprensani_016.csv,17.2
5,split\nicaragua_laprensani_015.csv,split\nicaragua_laprensani_015.csv,split\nicaragua,split\nicaragua_laprensani_015.csv,15.7
6,split\nicaragua_laprensani_014.csv,split\nicaragua_laprensani_014.csv,split\nicaragua,split\nicaragua_laprensani_014.csv,16.1
7,split\nicaragua_laprensani_013.csv,split\nicaragua_laprensani_013.csv,split\nicaragua,split\nicaragua_laprensani_013.csv,16.1
8,split\nicaragua_laprensani_012.csv,split\nicaragua_laprensani_012.csv,split\nicaragua,split\nicaragua_laprensani_012.csv,17.1
9,split\nicaragua_laprensani_011.csv,split\nicaragua_laprensani_011.csv,split\nicaragua,split\nicaragua_laprensani_011.csv,17.1


In [19]:
df.source.unique()

array(['split\\nicaragua'], dtype=object)

In [20]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 509.4


,size
source,
split\nicaragua,509.4


In [23]:
# Cleans the given files.
def basic_process(filename, output_folder):
    
    sizes = []
    
    df1 = pd.read_csv(filename, header=[0])
    sizes.append(df1.shape[0])
    
    if 'url' not in df1.columns:
        df1['url'] ='\n'
    if 'title' not in df1.columns:
        df1['title']='\n'
    
#     df1 = df1[~df1.url.isnull()]
    
    df1 = df1[~df1.text.isnull()]
    df1 = df1[~df1.text.duplicated()]
#     df1 = df1[~df1.url.duplicated()]
    
    df1.loc[df1.title.isnull(),'title']='\n'
    df1.loc[df1.url.isnull(),'url']='\n'
    if 'abstract' in df1:
        df1.loc[df1.abstract.isnull(),'abstract']='\n'
        
    # ------------------- Start cleaning  --------------------------#
    
    # Convert coding
    df1.text = df1.text.apply(lambda x: unicodetoascii(x))
    df1.text = df1.text.apply(lambda x: unicodedata.normalize("NFKD", x))
    
    # email
    df1.text = df1.text.apply(lambda x: re.sub("\S+@\S+(?:\.\S+)+",'',x))
    
    # telphone
    df1.text = df1.text.apply(lambda x: re.sub('\(\+( |-|\d)+\)( |-|\d)+',' ',x))
    df1.text = df1.text.apply(lambda x: re.sub('\+( |-|\d)+',' ',x))
    
    # noise
    df1.text =\
    df1.text.apply(lambda x: re.sub('\n(ad|advertisement|tweet):?\n', "", x, flags=re.IGNORECASE))
    
    # urls
    df1.text = df1.text.apply(lambda x: re.sub(r"http\S+", "", x))
    
    # delete too many \n
    df1.text = df1.text.apply(lambda x: re.sub('\n\n+', "\n\n", x, flags=re.IGNORECASE))
    
    # head and tails
    df1.text = df1.text.apply(lambda x: re.sub("^\s+|\s+$", "", x, flags=re.UNICODE)) 
    
    
    df1 = df1[df1.text.str.len()>100]
    
    sizes.append(df1.shape[0])
    
    # ------------------- Ending cleaning  --------------------------#
    
    filename = filename.split('/')[1]
    new_filename = output_folder + '/'+ filename

    print('%s:\t%s'%(filename, sizes))
    df1.to_csv(new_filename, index= False)
    return

In [24]:
# Takes the splits and cleans them.
pandarallel.initialize(nb_workers=8, progress_bar=False) 
_ = df['path'].apply(basic_process, output_folder='split')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
nicaragua_articulo66_000.csv:	[1389, 1371]
nicaragua_laprensani_019.csv:	[6966, 6929]
nicaragua_laprensani_018.csv:	[6966, 6241]
nicaragua_laprensani_017.csv:	[6966, 6703]
nicaragua_laprensani_016.csv:	[6966, 6765]
nicaragua_laprensani_015.csv:	[6966, 6573]
nicaragua_laprensani_014.csv:	[6966, 6352]
nicaragua_laprensani_013.csv:	[6967, 6434]
nicaragua_laprensani_012.csv:	[6967, 6653]
nicaragua_laprensani_011.csv:	[6967, 6770]
nicaragua_laprensani_010.csv:	[6967, 6872]
nicaragua_laprensani_009.csv:	[6967, 6965]
nicaragua_laprensani_008.csv:	[6967, 6955]
nicaragua_laprensani_007.csv:	[6967, 6558]
nicaragua_laprensani_006.csv:	[6967, 5013]
nicaragua_laprensani_005.csv:	[6967, 2326]
nicaragua_laprensani_004.csv:	[6967, 5650]
nicaragua_laprensani_003.csv:	[6967, 6091]
n

In [50]:

df = show_all_files('split')
df = df[~df.path.str.contains('filtered')]
df

,path,root,source,filename,size
0,split\nicaragua_articulo66_000.csv,split\nicaragua_articulo66_000.csv,split\nicaragua,split\nicaragua_articulo66_000.csv,5.5
1,split\nicaragua_laprensani_010.csv,split\nicaragua_laprensani_010.csv,split\nicaragua,split\nicaragua_laprensani_010.csv,18.9
3,split\nicaragua_laprensani_011.csv,split\nicaragua_laprensani_011.csv,split\nicaragua,split\nicaragua_laprensani_011.csv,17.3
5,split\nicaragua_laprensani_012.csv,split\nicaragua_laprensani_012.csv,split\nicaragua,split\nicaragua_laprensani_012.csv,17.2
7,split\nicaragua_laprensani_013.csv,split\nicaragua_laprensani_013.csv,split\nicaragua,split\nicaragua_laprensani_013.csv,16.1
9,split\nicaragua_laprensani_014.csv,split\nicaragua_laprensani_014.csv,split\nicaragua,split\nicaragua_laprensani_014.csv,16.0
12,split\nicaragua_laprensani_015.csv,split\nicaragua_laprensani_015.csv,split\nicaragua,split\nicaragua_laprensani_015.csv,15.8
13,split\nicaragua_laprensani_016.csv,split\nicaragua_laprensani_016.csv,split\nicaragua,split\nicaragua_laprensani_016.csv,17.4
15,split\nicaragua_laprensani_017.csv,split\nicaragua_laprensani_017.csv,split\nicaragua,split\nicaragua_laprensani_017.csv,17.5
17,split\nicaragua_laprensani_018.csv,split\nicaragua_laprensani_018.csv,split\nicaragua,split\nicaragua_laprensani_018.csv,18.6


In [36]:
print('total size:', df['size'].sum())
df.groupby(by=["source"], dropna=False).sum()

total size: 486.9


,size
source,
split\nicaragua,486.9


### Filter

In [54]:
# defines a metric for number of relevant and irrelevant words.
def find_relevant(str):
    return len(relevant_rules.findall(str))
def find_irelevant(str):
    return len(irelevant_rules.findall(str))

In [55]:
# find_relevant = lambda x: Counter(relevant_rules.findall(x.lower()))
# find_exclude = lambda x: Counter(irelevant_rules.findall(x.lower()))
#

# for each file, open and read into dataframe, then make sure text is NOT null
def filter(filepath):
    sizes = []
    df1 = pd.read_csv(filepath, header=[0])
    sizes.append(df1.shape[0]) 
    df1 = df1[~df1.text.isnull()]
    df1['relevant'] = df1['text'].apply(find_relevant)
    df1['irrelevant'] = df1['text'].apply(find_irelevant)
    relevant_count = df1.relevant
    irrelevant_count = df1.irrelevant
# Filter Calculation: Mean and SD of both relevant and irrelevant columns
    relevant_mean = np.mean(df1['relevant'])
    irrelevant_mean = np.mean(df1['irrelevant'])
    relevant_sd = np.std(df1['relevant'])
    irrelevant_sd = np.std(df1['irrelevant'])
    # print(df1['relevant'].describe())
    # print(relevant_mean, relevant_sd)
    # print(df1['irrelevant'].describe())
    # print(irrelevant_mean, irrelevant_sd)
# The next three lines are the filter, can be changed based on the needs of the project. Currently, throw out all rows with more than two irrelevant words/regex
    threshold = 1.5
    select_index = (relevant_count >= irrelevant_count) & (irrelevant_count < irrelevant_mean + threshold * irrelevant_sd) & (relevant_count > relevant_mean - threshold * relevant_sd)
    df1 = df1[select_index]   
    df1 = df1.drop('relevant', axis=1)
    df1 = df1.drop('irrelevant', axis=1)
    sizes.append(df1.shape[0])
# next line is debugging    
    # print(df1)
# create a filtered file with the given dataframe, will appear in same folder as unfiltered

    df1.to_csv(filepath[:-4] + '_filtered.csv', index=False)
    print('%s:\t%s'%(filepath[6:-4], sizes))

In [56]:
pandarallel.initialize(nb_workers=8, progress_bar=False) 
_ = df['path'].apply(filter)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
nicaragua_articulo66_000:	[1371, 1138]
nicaragua_laprensani_010:	[6872, 5214]
nicaragua_laprensani_011:	[6770, 5319]
nicaragua_laprensani_012:	[6653, 5071]
nicaragua_laprensani_013:	[6434, 4974]
nicaragua_laprensani_014:	[6352, 5275]
nicaragua_laprensani_015:	[6573, 5196]
nicaragua_laprensani_016:	[6765, 5493]
nicaragua_laprensani_017:	[6703, 5201]
nicaragua_laprensani_018:	[6241, 4848]
nicaragua_laprensani_019:	[6929, 5226]
nicaragua_nicaraguaactual_000:	[6781, 5505]
nicaragua_laprensani_009:	[6965, 4968]
nicaragua_laprensani_008:	[6955, 5199]
nicaragua_confidencial.digital_000:	[3766, 2981]
nicaragua_confidencial.digital_001:	[3766, 3285]
nicaragua_lajornadanet_000:	[87, 73]
nicaragua_laprensani_000:	[5565, 4289]
nicaragua_laprensani_001:	[6913, 5838]
nicaragua_l

# save to json

In [29]:
# folder = 'none'
# df = pd.DataFrame(columns=['path','source', 'filename'])
# df.path = [str(x) for x in pathlib.Path('%s/*'%folder).glob('**/*')]
df = pd.DataFrame(glob.glob('*/*'), columns = ['path'])
df = df[df.path.str.contains('_filtered')]
# df['root'] = df.path.apply(lambda x: x.split('/')[0])

df['source'] = df.path.apply(lambda x: x.split('/')[1][0:x.index('_')-6])
df['filename'] = df.path.apply(lambda x: x.split('/')[-1])

# df = df.sort_values('source').reset_index(drop=True)
# df['size'] = df['path'].parallel_apply(get_csv_size)
# df['filename'] = df.path.apply(lambda x: x.replace('%s/'%folder,''))

df['json_file'] = df['filename'].apply(lambda x: x.replace('.csv', '.json'))

df['json_file'] = df.source +'/'+ df.json_file
df['tar_file'] = 'tar/'+ df.json_file+'.tar.gz'
df.json_file = 'json/'+ df.json_file
df

,path,source,filename,json_file,tar_file
3,split\diaadia_000_filtered.csv,diaadia,diaadia_000_filtered.csv,json\diaadia\diaadia_000_filtered.json,tar\diaadia\diaadia_000_filtered.json.tar.gz
5,split\diaadia_001_filtered.csv,diaadia,diaadia_001_filtered.csv,json\diaadia\diaadia_001_filtered.json,tar\diaadia\diaadia_001_filtered.json.tar.gz


In [28]:
os.mkdir(cwd + '/json/')
os.mkdir(cwd + '/tar/')

for i in df.source.unique():
    os.mkdir(cwd + '/json/'+i)
    os.mkdir(cwd + '/tar/'+i)  

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'c:\\Users\\vikas\\Documents\\GitHub\\conflibert-spanish\\Filter\\json\\'

In [30]:
df1 = pd.read_csv(df.path[3], header=[0])
df1

,Unnamed: 0,page,rows,link,text,url,title
0,1,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/Ucrania%20DD.jpg?itok=laMENc2k"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758"">\n<h5 class=""title-fixed der"">Donan 500.000 euros a ACNUR para...",https://www.diaadia.com.pa/mundo/donan-500000-euros-acnur-para-ayuda-humanitaria-en-ucrania-751758,"Brother Industries, Ltd. anunció que el grupo donará 500.000 euros a ACNUR (Alto Comisionado de las Naciones Unidas para los Refugiados).\n\nEsta donación se entregará a través de ACNUR para apoyar los esfuerzos de ayuda humanitaria en Ucrania y los países vecinos.\n\nEl corporativo (proveedor en tecnología de equipos de oficina) lamenta enormemente que tantas vidas inocentes se vean afectadas negativamente por la situación actual en Europa del Este y ha decidido extender su apoyo a los necesitados.\n\nEsperan que todos los afectados estén a salvo y agregaron que sus pensamientos ...",\n,\n
1,3,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/quiero-seguridad-pide-al-congreso-de-eeuu-una-nina-sobreviviente-del-tiroteo-de-uvalde-751506"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/cerillo.jpg?itok=bbKqpAEt"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/quiero-seguridad-pide-al-congreso-de-eeuu-una-nina-sobreviviente-del-tiroteo-de-uvalde-751506"">\n<h5 class...",https://www.diaadia.com.pa/mundo/quiero-seguridad-pide-al-congreso-de-eeuu-una-nina-sobreviviente-del-tiroteo-de-uvalde-751506,"Miah Cerrillo, una niña de 11 años que sobrevivió al tiroteo en la escuela de Uvalde (Texas, EE.UU.), reclamó ayer miércoles ""seguridad"" al Congreso y describió la pesadilla de aquel día, cuando tuvo que cubrirse con la sangre de un amigo para hacerse la muerta y salvar la vida.\n\nCerrillo intervino en video ante un comité de la Cámara Baja de EE.UU., ante el que testificaron otras víctimas del tiroteo de Uvalde, en el que murieron 19 alumnos y dos profesoras, así como víctimas del ataque racista en Buffalo (Nueva York) que se cobró la vida de diez personas afroamericanas.\n\...",\n,\n
2,4,https://www.diaadia.com.pa/mundo?page=0,"<div class=""col-12 col-md-6"">\n<div class="" mb-3 card_secciones "">\n<div class=""card"">\n<a href=""/mundo/los-casos-globales-de-covid-bajan-un-12-semanalmente-y-las-muertes-un-22-751457"">\n<img src="" /sites/default/files/styles/secciones/public/2022-06/hisopadas.jpg?itok=Zn00UTOW"" width=""100%""/>\n</a>\n<div class=""card-body-fixed"">\n<div class=""card-title custom"">\n<div class=""bg-blanco"">\n<span class=""categoria-bg"">Mundo </span>\n</div>\n<a class=""hovernaranja"" href=""/mundo/los-casos-globales-de-covid-bajan-un-12-semanalmente-y-las-muertes-un-22-751457"">\n<h5 class=""title-fixed der"">Los cas...",https://www.diaadia.com.pa/mundo/los-casos-globales-de-covid-bajan-un-12-semanalmente-y-las-muertes-un-22-751457,"El mundo registró 3 millones de casos de covid la semana pasada, un descenso del 12 % con respecto a los siete días anteriores, mientras que las muertes bajaron un 22 %, hasta 7.600, indicó el nuevo informe epidemiológico de la Organización Mundial de la Salud (OMS).\n\nEn América, la región con más casos en cifras absolutas (1,1 millones), la bajada fue de sólo el 1 %, mien

In [31]:
def convert_json_tar(filename, json_file, tar_file):
    df1 = pd.read_csv(filename, header=[0])      
    df1.to_json(json_file, orient="records", lines=True)
    
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(json_file, arcname=os.path.basename(json_file))
df.apply(lambda x: convert_json_tar(x['path'], x['json_file'], x['tar_file']), axis=1)

3    None
5    None
dtype: object

In [32]:
pandarallel.initialize() 

df.source = df.path.apply(lambda x: x.split('/')[1][0:x.index('_')-6]).values
df['size'] = df['path'].apply(get_csv_size)
df

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,path,source,filename,json_file,tar_file,size
3,split\diaadia_000_filtered.csv,diaadia,diaadia_000_filtered.csv,json\diaadia\diaadia_000_filtered.json,tar\diaadia\diaadia_000_filtered.json.tar.gz,17.5
5,split\diaadia_001_filtered.csv,diaadia,diaadia_001_filtered.csv,json\diaadia\diaadia_001_filtered.json,tar\diaadia\diaadia_001_filtered.json.tar.gz,12.6


ValueError: '_' is not in list

In [33]:
print(df['size'].sum())

df.groupby(by=["source"], dropna=False).sum()

30.1


,size
source,
diaadia,30.1
